In [10]:
import pandas as pd
from tqdm import tqdm
from itables import show as table_show

In [11]:
ENTREPOT_PATH = '~/Bureau/utils/data/'
METEO_PATH = '~/Bureau/utils/data/meteo/'
SPATIAL_PATH = './data/spatial/'

In [12]:
studied_ids = [
    'fr.inra.agrosyst.api.entities.effective.EffectiveIntervention_0081a991-49bf-422e-b1bf-c931c307acb9',
    'fr.inra.agrosyst.api.entities.effective.EffectiveIntervention_02f2297a-8673-42dc-9365-3112c48847db',
    'fr.inra.agrosyst.api.entities.effective.EffectiveIntervention_03d47a0f-fc9e-467d-b807-0277acad258d',
    'fr.inra.agrosyst.api.entities.effective.EffectiveIntervention_5fd3e685-b200-4879-b5dc-ef3239326e9e'
]

In [13]:
donnees = {}

def import_df(df_name, path_data, sep, index_col=None):
    donnees[df_name] = pd.read_csv(path_data+df_name+'.csv', sep = sep, index_col=index_col, low_memory=False)

def import_dfs(df_names, path_data, sep = ',', index_col=None, verbose=False):
    for df_name in tqdm(df_names) : 
        if(verbose) :
            print(" - ", df_name)
        import_df(df_name, path_data, sep, index_col=index_col)

tables_entrepot = [
    'combinaison_outil', 'composant_parc_materiel', 'intervention_realise', 'materiel'
]

tables_performance = [
]

# import des données de l'entrepôt avec la colonne 'id' en index 
import_dfs(tables_entrepot, ENTREPOT_PATH, sep = ',',index_col='id', verbose=False)

# import des données de performance
import_dfs(['combinaison_outil_composant_parc_materiel'], ENTREPOT_PATH, sep = ',', verbose=False)


100%|██████████| 1/1 [00:00<00:00, 1368.90it/s]


In [14]:
donnees['intervention_realise_test'] = donnees['intervention_realise'].loc[
    donnees['intervention_realise'].index.isin(studied_ids)
]

donnees['combinaison_outil_test'] = donnees['combinaison_outil'].loc[
    donnees['combinaison_outil'].index.isin(donnees['intervention_realise_test']['combinaison_outil_id'])
]

donnees['combinaison_outil_composant_parc_materiel_test'] = donnees['combinaison_outil_composant_parc_materiel'].loc[
    donnees['combinaison_outil_composant_parc_materiel']['combinaison_outil_id'].isin(donnees['combinaison_outil_test'].index)
]

donnees['composant_parc_materiel_test'] = donnees['composant_parc_materiel'].loc[
    donnees['composant_parc_materiel'].index.isin(donnees['combinaison_outil_composant_parc_materiel_test']['composant_parc_materiel_id']) |
    donnees['composant_parc_materiel'].index.isin(donnees['combinaison_outil_test']['tracteur_composant_parc_materiel_id'])
]

donnees['materiel_test'] = donnees['materiel'].loc[
    donnees['materiel'].index.isin(donnees['composant_parc_materiel_test']['materiel_id'])
]

In [ ]:
path='/home/utilisateur/Documents/catalogue_script_agrosyst/02_outils/tests/data/test_get_intervention_realise_combinaison_outils_can/'
donnees['intervention_realise_test'].to_csv(path+'intervention_realise'+'.csv')
donnees['materiel_test'].to_csv(path+'materiel'+'.csv')
donnees['combinaison_outil_test'].to_csv(path+'combinaison_outil'+'.csv')
donnees['combinaison_outil_composant_parc_materiel_test'].to_csv(path+'combinaison_outil_composant_parc_materiel'+'.csv')
donnees['composant_parc_materiel_test'].to_csv(path+'composant_parc_materiel'+'.csv')